# IMPORT LIBS

In [1]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 39 (delta 8), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.
PLEASE READ
********************************************************************************************************
Colab Notebooks Migration Notice:
 
We have changed the location of the Colab script to our new CSP utilties repo.
We are also dropping support for versions below 0.11, as we are on 0.13 nightlies
 
Not all Colab notebooks are updated (like personal Colabs) and/or are using outdated versions of RAPIDS,
so we HIGHLY encourage users to update their scripts and Colab notebooks both with the new link AND with 
to the new API.  Otherwise, your code MAY BREAK
 
This notice will disappear/be updated on our next release.
 
Please enter in the box your desired RAPIDS version (ex: '0.11' or '0.12', between 0.11 to

In [2]:
!ls /opt/

bin  google  nvidia


In [3]:
! ls /opt/nvidia/

nsight-compute	nsight-systems


In [0]:
from cuml import ExponentialSmoothing as cuES
from numba import cuda
import time
import cudf
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing as smES
from sklearn.metrics import r2_score
from time import time

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
#Install PYSPARK - Runs on GPU and TPU.  
!pip install pyspark
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# global setup to work around with pandas udf
!sudo pip3 install pyarrow==0.14.1
# see answers here https://stackoverflow.com/questions/58458415/pandas-scalar-udf-failing-illegalargumentexception
os.environ["ARROW_PRE_0_15_IPC_FORMAT"] = '1'

#Install pandas SQL
!pip install pandasql
!pip install pyarrow

     |████████████████████████████████| 217.8 MB 2.9 kB/s 
     |████████████████████████████████| 197 kB 51.3 MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=ae0894cf384263a7c5f97d8f6018385f4fffda69cb0b1ee5cbc84d8d54ba77ef
  Stored in directory: /root/.cache/pip/wheels/84/30/e3/c51c5cd0229631e662d29d7b578a3e5949a4c8db033ffb70aa
Successfully built pyspark
     / 19.8 MB 113 kB/s
     |████████████████████████████████| 12.4 MB 4.9 MB/s 
     |████████████████████████████████| 8.7 MB 34.6 MB/s 
     |████████████████████████████████| 607 kB 66.6 MB/s 
     |████████████████████████████████| 6.5 MB 56.1 MB/s 
     |████████████████████████████████| 1.5 MB 78.2 MB/s 
     |████████████████████████████████| 59 kB 8.5 MB/s 
     |████████████████████████████████| 121 kB 84.1 MB/s 
     |████████████████████████████████| 90 kB 14.4 MB/s 
     |████████████████████████████████| 1.6 MB 47.9 MB/s 
     |████████████████████████████████| 231 kB

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)
     |████████████████████████████████| 6.1 MB 8.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26817 sha256=a3d52abe38d395a7d84e5b9c92cfe9e5f0c820f2713701173e517d75a6f28928
  Stored in directory: /root/.cache/pip/wheels/a6/64/11/62d5f7b88421a5d80068ac9937fe2915e497da3eba900c8f59
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.15-cp36-cp36m-linux_x86_64.whl size=1230188 sha256=8da777e2a3aa61fa3cf3d62046ea4c81fa80bc7620789db95bf33a9cee1b06a1
  Stored in directory: /root/.cache/pip/wheels/4a/1b/3a/c73044d7be48baeb47cbee343334f7803726ca1e9ba7b290

In [0]:
#All the required imports.  
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import pandasql as pdsql
import glob
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt  # 2D plotting
import seaborn as sns  # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING



#SPARK STUFF
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import pandas_udf, PandasUDFType, sum, max, col, concat, lit
from pyspark.sql.functions import col
import pyarrow

spark = (SparkSession
             .builder
             .appName("forecasting")
             .config('spark.sql.execution.arrow.enable', 'true')
             .getOrCreate()
             )

#PICKLE
import pickle

#TEST LOAD

In [0]:
cols = ['TRANSACTION_DATE', 'BUYER_ZIP', 'BUYER_COUNTY','DRUG_NAME',  'MME', 'BUYER_NAME', 'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY']


In [0]:
hydro = cudf.read_csv('/content/drive/My Drive/IST_718_Project/raw_by_drug_data/HYDROCODONE.csv',sep='|',columns=cols)

In [0]:
oxy = cudf.read_csv('/content/drive/My Drive/IST_718_Project/raw_by_drug_data/OXYCODONE.csv',sep='|',columns=cols)

# Obtain - Load Data from Parquet

In [0]:
#Read the parquet file.  
parquetFile = spark.read.parquet("/content/drive/My Drive/IST_718_Project/parquet/clean_drugs.parquet")

#Temp view for stats
parquetFile.createOrReplaceTempView("cleandrugs")

# Spark

In [0]:
#check the data via spark
#SQL statement for walking through each zip code, summing the MME per day and submitting
#to FB Prophet.  
sql_statement = '''
  SELECT BUYER_ZIP, TRANSACTION_DATE, sum(MMEperDay) as MMEperDay
  FROM cleandrugs
  GROUP BY BUYER_ZIP, TRANSACTION_DATE
  ORDER BY BUYER_ZIP, TRANSACTION_DATE
  '''

sc = spark.sparkContext

df = (
  spark
    .sql( sql_statement )
    .repartition(sc.defaultParallelism, ['BUYER_ZIP'])
  ).cache()

In [0]:
df.show()

+-----+-------------------+------------------+
|  Zip|                 ds|                 y|
+-----+-------------------+------------------+
|10001|2007-01-31 00:00:00| 3173.663595612904|
|10001|2007-02-28 00:00:00|  6378.32285357143|
|10001|2007-03-31 00:00:00| 2849.208812419354|
|10001|2007-04-30 00:00:00|3314.0542903333353|
|10001|2007-05-31 00:00:00| 3355.574336612904|
|10001|2007-06-30 00:00:00| 3271.836667966667|
|10001|2007-07-31 00:00:00| 4300.673245322581|
|10001|2007-08-31 00:00:00| 4706.262178709679|
|10001|2007-09-30 00:00:00|3441.4539563333337|
|10001|2007-10-31 00:00:00| 4880.645777258063|
|10001|2007-11-30 00:00:00|       6313.275948|
|10001|2007-12-31 00:00:00|2557.6453790645164|
|10001|2008-01-31 00:00:00| 5316.437748354837|
|10001|2008-02-28 00:00:00| 6308.028111964288|
|10001|2008-03-31 00:00:00| 5585.575010580643|
|10001|2008-04-30 00:00:00| 7827.972514800001|
|10001|2008-05-31 00:00:00|4155.0657009032275|
|10001|2008-06-30 00:00:00| 4706.336596800001|
|10001|2008-0

In [0]:
#Pivot table by zip.
df = df.groupBy("TRANSACTION_DATE").pivot("BUYER_ZIP").sum("MMEperDay")
df = df.orderBy("TRANSACTION_DATE")
df.show()

+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----

#Spark to Rapids hand-off

In [0]:
# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [0]:
data = cudf.DataFrame.from_pandas(df.select("*").toPandas())
data.to_pandas().head()

,TRANSACTION_DATE,10001,10002,10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10016,10017,10018,10019,10020,10021,10022,10023,10024,10025,10026,10027,10028,10029,10030,10031,10032,10033,10034,10035,10036,10037,10038,10039,10040,10044,...,14727,14731,14733,14735,14737,14738,14739,14740,14750,14757,14760,14770,14772,14779,14781,14787,14801,14802,14810,14813,14814,14823,14830,14837,14840,14843,14845,14850,14853,14865,14870,14886,14891,14892,14895,14901,14902,14903,14904,14905
0,2007-01-31,3173.663596,12858.556436,60666.110216,389.062487,1272.424266,307.572486,571.855538,36840.380471,95776.519645,50128.952370,4407.656766,14893.077998,16498.739696,25402.146401,5264.013698,3158.476348,105637.650769,1473.269733,100381.919390,20088.508485,27187.401381,17534.315083,51537.553867,7395.902108,18319.661791,91445.138754,66563.192199,450.133145,21776.344534,26900.416601,12943.891089,7486.959371,70073.402591,7024.571433,5983.115381,28109.929012,5887.846269,2641.411140,1635.228658,...,5893.596468,764.236331,4049.671773,4507.899735,1950.311468,568.591187,810.353497,NaN,5330.321237,4245.353445,17724.062560,201.911814,1258.532614,10050.894521,2131.270161,4940.864809,NaN,663.987097,10262.838837,2500.773929,NaN,794.229659,24983.667134,922.484855,1178.475224,12568.140842,19484.683035,58980.363282,200.686646,5799.315843,2740.497852,3142.202377,4988.807189,2436.317416,19853.954866,10319.131202,718.718226,1583.434994,17686.658635,7639.883248
1,2007-02-28,6378.322854,11628.546271,52087.591344,822.101876,3581.825227,1793.412800,604.767753,47855.118344,84125.129533,47205.837476,8249.334968,13271.571969,18155.595100,38574.862415,10607.451353,4605.192082,153969.510058,2724.126666,120424.881884,32996.842692,28701.328215,17796.970964,51340.046686,10504.816827,22790.082141,112887.601285,64473.471837,726.742854,16169.120323,20949.660339,10307.787729,5841.744067,71230.845114,13138.907944,16941.700227,40352.355658,2760.279446,4236.857013,687.607716,...,8165.448956,619.549056,6876.296350,2291.398513,1569.637104,685.312800,421.599407,NaN,5707.042704,3229.552979,20558.023429,409.067254,837.830357,8751.326144,5099.833233,6140.035805,NaN,690.138194,9793.770591,1737.819943,NaN,648.642857,22170.998335,789.182143,374.105357,12596.952764,16661.000825,59908.441154,372.301339,7581.944689,3173.372175,4158.116751,5872.430477,2083.928709,22451.934785,8821.282035,367.564286,1167.587798,18037.125663,8050.293871
2,2007-03-31,2849.208812,17625.872761,58937.151003,2190.142470,2000.513014,1870.420517,5045.450000,43852.480684,76511.405760,53951.517104,8423.400740,15878.298156,19973.926452,33471.202655,10319.530310,8612.246646,61798.684042,4167.398211,139803.357953,44838.204482,27597.299689,21662.872828,67105.532840,11661.321722,19259.939050,131669.609924,55191.258762,1161.149330,19485.216063,27338.882603,13895.294375,7257.720362,35518.548895,12371.467487,14119.632639,42687.015362,4766.240011,3174.646871,2785.639226,...,7189.477463,758.911507,6725.423121,3186.940732,1875.001791,467.380288,512.646106,NaN,5562.207000,3823.471182,21024.663180,475.318266,1319.824821,8089.210272,1112.928629,6265.976797,NaN,678.633871,8164.228987,1648.248658,NaN,945.048929,24350.429098,732.338710,1043.684855,10041.180826,19073.770244,55927.548095,288.829210,9237.054458,3302.108334,3772.516717,6449.608467,2605.504723,28016.306344,10995.749635,512.290501,1769.033421,13098.725806,8993.260122
3,2007-04-30,3314.054290,11617.441162,39929.718825,354.321384,1056.582830,159.743750,387.388330,42819.324786,81716.073951,47170.897945,3771.859442,15898.129302,13426.148181,31890.247249,3805.048916,2255.094049,53323.243730,891.869615,114170.517408,17895.382265,17574.029324,14999.593987,41933.326398,10512.214198,18902.321608,23433.541100,43669.300908,646.909217,20049.706751,24504.489054,11423.632275,5942.726680,55944.542743,7575.190983,6531.420629,25611.555430,3275.514001,2436.321084,824.435000,...,5627.560783,968.571767,5143.951321,2784.816810,1763.430000,438.644351,715.722500,NaN,4694.992411,7539.326447,

In [0]:
data.dtypes

TRANSACTION_DATE    datetime64[ns]
10001                      float64
10002                      float64
10003                      float64
10004                      float64
                         ...      
14901                      float64
14902                      float64
14903                      float64
14904                      float64
14905                      float64
Length: 878, dtype: object

# Rapids

In [0]:
data = data.set_index(['TRANSACTION_DATE'],drop=True)

In [0]:
#Pull out the columns for reassignment after the forecast
df_columns = data.columns

In [0]:
#view sampling
data.head()

,10001,10002,10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10016,10017,10018,10019,10020,10021,10022,10023,14813,14814,14823,14830,14837,14840,14843,14845,14850,14853,14865,14870,14886,14891,14892,14895,14901,14902,14903,14904,14905
TRANSACTION_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-01-31,3173.663596,12858.556436,60666.110216,389.062487,1272.424266,307.572486,571.855538,36840.380471,95776.519645,50128.952370,4407.656766,14893.077998,16498.739696,25402.146401,5264.013698,3158.476348,105637.650769,1473.269733,100381.919390,20088.508485,27187.401381,2500.773929,null,794.229659,24983.667134,922.484855,1178.475224,12568.140842,19484.683035,58980.363282,200.686646,5799.315843,2740.497852,3142.202377,4988.807189,2436.317416,19853.954866,10319.131202,718.718226,1583.434994,17686.658635,7639.883248
2007-02-28,6378.322854,11628.546271,52087.591344,822.101876,3581.825227,1793.412800,604.767753,47855.118344,84125.129533,47205.837476,8249.334968,13271.571969,18155.595100,38574.862415,10607.451353,4605.192082,153969.510058,2724.126666,120424.881884,32996.842692,28701.328215,1737.819943,null,648.642857,22170.998335,789.182143,374.105357,12596.952764,16661.000825,59908.441154,372.301339,7581.944689,3173.372175,4158.116751,5872.430477,2083.928709,22451.934785,8821.282035,367.564286,1167.587798,18037.125663,8050.293871
2007-03-31,2849.208812,17625.872761,58937.151003,2190.142470,2000.513014,1870.420517,5045.450000,43852.480684,76511.405760,53951.517104,8423.400740,15878.298156,19973.926452,33471.202655,10319.530310,8612.246646,61798.684042,4167.398211,139803.357953,44838.204482,27597.299689,1648.248658,null,945.048929,24350.429098,732.338710,1043.684855,10041.180826,19073.770244,55927.548095,288.829210,9237.054458,3302.108334,3772.516717,6449.608467,2605.504723,28016.306344,10995.749635,512.290501,1769.033421,13098.725806,8993.260122
2007-04-30,3314.054290,11617.441162,39929.718825,354.321384,1056.582830,159.743750,387.388330,42819.324786,81716.073951,47170.897945,3771.859442,15898.129302,13426.148181,31890.247249,3805.048916,2255.094049,53323.243730,891.869615,114170.517408,17895.382265,17574.029324,1776.562500,null,936.190560,21530.627858,887.920000,1109.535000,13665.224376,14742.932445,57293.095015,483.381851,4562.792020,3038.264351,2804.625950,7092.127816,2749.050031,17098.952695,7933.854933,546.468750,2181.595183,16394.733811,6496.801931
2007-05-31,3355.574337,9235.305108,46702.392291,682.414683,1380.500537,469.472759,458.355916,48198.253853,94026.396923,42866.250405,3966.106917,14021.651290,17359.581959,30453.470338,4241.827190,3284.099131,66858.413350,879.811191,121350.765498,23589.988981,23797.511486,870.923836,null,814.282529,21311.728742,947.158065,1240.848658,14788.346722,19869.219185,58407.451166,286.369355,6503.134723,2639.409049,4566.543633,7791.806885,3693.466519,20517.980701,10691.512551,739.212097,1999.706226,19928.157936,7564.818966


In [0]:
data.shape

(96, 877)

In [0]:
data = data.fillna(0)

In [0]:
%%time
months_to_predict = 12
cu_hw = cuES(data, seasonal='multiplicative', seasonal_periods=12, ts_num=877)
cu_hw.fit()
cu_preds = cu_hw.forecast(months_to_predict)
cu_hw_forecast = cu_preds

CPU times: user 3 s, sys: 584 ms, total: 3.59 s
Wall time: 3.59 s


In [0]:
cu_hw_forecast.head(12)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876
0,12485.157050,17628.147839,82667.569831,723.589167,2376.932823,140.054489,8103.350089,22669.597818,118850.284509,124850.371064,12304.075597,27662.457975,10814.082940,85482.600057,42842.490299,9481.031076,74752.760406,427.725911,68473.050625,22997.732856,19321.731237,3025.825303,NaN,3651.446797,31401.954023,1944.221462,2698.448089,16047.456301,36212.811091,73882.843113,27.682385,11983.737016,8717.759805,10296.040837,17926.241413,15633.827887,20335.750968,27375.923304,3816.444719,11196.811808,43494.636351,10756.087654
1,11390.503124,18651.775737,85103.890314,834.181013,1952.754594,140.552689,8603.665705,23088.669952,130403.322767,122092.183858,12774.626474,31453.043235,11010.602661,88422.258158,46191.330586,7968.979602,73734.038170,449.777456,72688.703408,28196.931144,18490.990882,2443.300983,NaN,3478.714550,27330.887193,1614.674317,2310.530358,16911.152445,38224.009876,73897.870702,32.083520,9390.716091,9118.417341,9551.396295,19749.386736,14339.049654,20308.750152,26909.903908,3284.928749,11006.335147,44000.768243,10730.033002
2,11871.406679,19229.865588,82760.101182,870.630039,1650.823531,86.150598,9471.864321,20860.790383,120629.236826,123827.703501,11993.030077,26101.465700,9253.607706,86127.492249,41971.763148,7227.962244,73507.279092,468.409965,65759.901184,24054.869852,19116.524039,2552.934897,NaN,3334.369552,32119.375558,1931.260115,2346.312469,17469.883250,39660.765021,74629.572291,27.039114,9010.404781,8026.172019,10117.329197,19237.851453,6869.941440,18758.191532,26671.914208,3507.862331,11687.469339,44311.341572,10165.611051
3,12684.690942,18874.320935,91107.543791,643.944864,1668.916510,112.361195,10888.581190,22587.152171,109054.648414,120171.863986,12749.318975,26501.180137,11662.606601,96657.376383,43309.422828,7634.551631,75963.590928,386.979221,74538.358344,21140.990015,17870.652922,2520.866549,NaN,4283.178430,31358.168795,1773.851532,2157.031396,15687.182453,36969.374785,72802.237211,30.945573,9433.840038,8082.015386,10103.078679,19658.333095,13078.767387,19023.387263,25406.996582,3299.423588,11227.604503,42364.223066,9211.598572
4,12808.608383,17574.395043,85201.014279,613.483661,1684.273768,98.702635,10414.581340,21109.245603,117950.183470,123045.055028,11645.698077,26651.425866,12746.614040,98239.196215,50223.325645,7123.894427,80883.153600,381.158724,59267.374918,22498.985839,17490.593658,2156.782412,NaN,3648.291618,29540.986068,1744.633059,1421.613710,16798.816001,36965.448274,73407.698124,19.251443,9946.781998,8243.253903,9462.255185,20699.967708,7151.283060,18398.552585,28221.728117,2904.670402,10707.167004,41779.601595,8978.626587
5,11405.951549,18966.704506,78587.289560,745.760282,1417.869984,106.313709,14682.229730,19959.882236,118279.257931,123660.745772,11876.924037,24264.372521,12886.648500,91526.676723,51240.287253,7350.826016,80567.351656,330.142062,62196.030103,20981.078423,17244.808049,2685.803179,NaN,3278.240652,31710.838857,1956.412628,1377.145087,16735.823275,40039.500454,72983.917455,16.573856,10514.382536,9429.275855,10302.012915,24054.789500,10933.690214,18751.338868,26887.030115,2666.172136,15062.035685,42815.237900,9783.694701
6,16183.865060,17124.024255,81513.643541,451.009491,1563.918324,46.257938,16764.582843,21347.274025,134358.265439,136147.683493,11869.943872,28465.703047,11832.023577,103597.576057,68594.164181,6982.189742,82640.576889,190.334521,70060.468165,26505.645547,17254.193466,2687.181130,NaN,4291.976886,32613.583271,2264.423760,3448.307234,16950.030925,41750.162415,74771.131375,13.121706,9520.936395,11145.478267,10094.892586,21467.814641,11389.563655,20197.339619,29545.188253,3099.531255,10780.407150,48129.840251,9648.519471
7,14879.681663,18004.091542,71596.060895,617.718893,1605.619051,65.776345,16110.858293,20962.790815,121217.991288,118645.430088,10380.902974,25703.277039,12959.951893,91726.055087,57498.642559,7395.444772,79086.8

In [0]:
cu_hw_forecast.shape

(12, 877)

In [0]:
cu_hw_forecast.columns = df_columns

In [0]:
cu_hw_forecast.head()

,10001,10002,10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10016,10017,10018,10019,10020,10021,10022,10023,14813,14814,14823,14830,14837,14840,14843,14845,14850,14853,14865,14870,14886,14891,14892,14895,14901,14902,14903,14904,14905
0,12485.157050,17628.147839,82667.569831,723.589167,2376.932823,140.054489,8103.350089,22669.597818,118850.284509,124850.371064,12304.075597,27662.457975,10814.082940,85482.600057,42842.490299,9481.031076,74752.760406,427.725911,68473.050625,22997.732856,19321.731237,3025.825303,NaN,3651.446797,31401.954023,1944.221462,2698.448089,16047.456301,36212.811091,73882.843113,27.682385,11983.737016,8717.759805,10296.040837,17926.241413,15633.827887,20335.750968,27375.923304,3816.444719,11196.811808,43494.636351,10756.087654
1,11390.503124,18651.775737,85103.890314,834.181013,1952.754594,140.552689,8603.665705,23088.669952,130403.322767,122092.183858,12774.626474,31453.043235,11010.602661,88422.258158,46191.330586,7968.979602,73734.038170,449.777456,72688.703408,28196.931144,18490.990882,2443.300983,NaN,3478.714550,27330.887193,1614.674317,2310.530358,16911.152445,38224.009876,73897.870702,32.083520,9390.716091,9118.417341,9551.396295,19749.386736,14339.049654,20308.750152,26909.903908,3284.928749,11006.335147,44000.768243,10730.033002
2,11871.406679,19229.865588,82760.101182,870.630039,1650.823531,86.150598,9471.864321,20860.790383,120629.236826,123827.703501,11993.030077,26101.465700,9253.607706,86127.492249,41971.763148,7227.962244,73507.279092,468.409965,65759.901184,24054.869852,19116.524039,2552.934897,NaN,3334.369552,32119.375558,1931.260115,2346.312469,17469.883250,39660.765021,74629.572291,27.039114,9010.404781,8026.172019,10117.329197,19237.851453,6869.941440,18758.191532,26671.914208,3507.862331,11687.469339,44311.341572,10165.611051
3,12684.690942,18874.320935,91107.543791,643.944864,1668.916510,112.361195,10888.581190,22587.152171,109054.648414,120171.863986,12749.318975,26501.180137,11662.606601,96657.376383,43309.422828,7634.551631,75963.590928,386.979221,74538.358344,21140.990015,17870.652922,2520.866549,NaN,4283.178430,31358.168795,1773.851532,2157.031396,15687.182453,36969.374785,72802.237211,30.945573,9433.840038,8082.015386,10103.078679,19658.333095,13078.767387,19023.387263,25406.996582,3299.423588,11227.604503,42364.223066,9211.598572
4,12808.608383,17574.395043,85201.014279,613.483661,1684.273768,98.702635,10414.581340,21109.245603,117950.183470,123045.055028,11645.698077,26651.425866,12746.614040,98239.196215,50223.325645,7123.894427,80883.153600,381.158724,59267.374918,22498.985839,17490.593658,2156.782412,NaN,3648.291618,29540.986068,1744.633059,1421.613710,16798.816001,36965.448274,73407.698124,19.251443,9946.781998,8243.253903,9462.255185,20699.967708,7151.283060,18398.552585,28221.728117,2904.670402,10707.167004,41779.601595,8978.626587


In [0]:
data.head()

,10001,10002,10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10016,10017,10018,10019,10020,10021,10022,10023,14813,14814,14823,14830,14837,14840,14843,14845,14850,14853,14865,14870,14886,14891,14892,14895,14901,14902,14903,14904,14905
TRANSACTION_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-01-31,3173.663596,12858.556436,60666.110216,389.062487,1272.424266,307.572486,571.855538,36840.380471,95776.519645,50128.952370,4407.656766,14893.077998,16498.739696,25402.146401,5264.013698,3158.476348,105637.650769,1473.269733,100381.919390,20088.508485,27187.401381,2500.773929,0.0,794.229659,24983.667134,922.484855,1178.475224,12568.140842,19484.683035,58980.363282,200.686646,5799.315843,2740.497852,3142.202377,4988.807189,2436.317416,19853.954866,10319.131202,718.718226,1583.434994,17686.658635,7639.883248
2007-02-28,6378.322854,11628.546271,52087.591344,822.101876,3581.825227,1793.412800,604.767753,47855.118344,84125.129533,47205.837476,8249.334968,13271.571969,18155.595100,38574.862415,10607.451353,4605.192082,153969.510058,2724.126666,120424.881884,32996.842692,28701.328215,1737.819943,0.0,648.642857,22170.998335,789.182143,374.105357,12596.952764,16661.000825,59908.441154,372.301339,7581.944689,3173.372175,4158.116751,5872.430477,2083.928709,22451.934785,8821.282035,367.564286,1167.587798,18037.125663,8050.293871
2007-03-31,2849.208812,17625.872761,58937.151003,2190.142470,2000.513014,1870.420517,5045.450000,43852.480684,76511.405760,53951.517104,8423.400740,15878.298156,19973.926452,33471.202655,10319.530310,8612.246646,61798.684042,4167.398211,139803.357953,44838.204482,27597.299689,1648.248658,0.0,945.048929,24350.429098,732.338710,1043.684855,10041.180826,19073.770244,55927.548095,288.829210,9237.054458,3302.108334,3772.516717,6449.608467,2605.504723,28016.306344,10995.749635,512.290501,1769.033421,13098.725806,8993.260122
2007-04-30,3314.054290,11617.441162,39929.718825,354.321384,1056.582830,159.743750,387.388330,42819.324786,81716.073951,47170.897945,3771.859442,15898.129302,13426.148181,31890.247249,3805.048916,2255.094049,53323.243730,891.869615,114170.517408,17895.382265,17574.029324,1776.562500,0.0,936.190560,21530.627858,887.920000,1109.535000,13665.224376,14742.932445,57293.095015,483.381851,4562.792020,3038.264351,2804.625950,7092.127816,2749.050031,17098.952695,7933.854933,546.468750,2181.595183,16394.733811,6496.801931
2007-05-31,3355.574337,9235.305108,46702.392291,682.414683,1380.500537,469.472759,458.355916,48198.253853,94026.396923,42866.250405,3966.106917,14021.651290,17359.581959,30453.470338,4241.827190,3284.099131,66858.413350,879.811191,121350.765498,23589.988981,23797.511486,870.923836,0.0,814.282529,21311.728742,947.158065,1240.848658,14788.346722,19869.219185,58407.451166,286.369355,6503.134723,2639.409049,4566.543633,7791.806885,3693.466519,20517.980701,10691.512551,739.212097,1999.706226,19928.157936,7564.818966


#Save

In [0]:
path = '/content/drive/My Drive/IST_718_Project/Forecast_Results/forecast_model_rapids_holtwinters.csv'
cu_hw_forecast.to_pandas().to_csv(path,index=False)